## Step1   Estimate Daily Volatilities with EWMA Model

In [57]:
# this function wrote for calculating daily volatilities
# using EWMA (Exponentially Weighted Moving Average) model


def ewma_vol(data, observed_days, decay_factor):
    """
    data type: dataframe(dataframe contains date, price and log-return)
    observed_days type: int(observed days)
    decay_factor type: float(decay factor)
    output type: dataframe(dataframe contains date, price, log-return and volatilitis)
    """
    # set the first vol using root mean square of return
    data.loc[(observed_days - 1), 'Volatility'] = np.sqrt(
        ((data.loc[:observed_days - 1, 'Log Return'] - 0) ** 2).mean())
    # calculate vol using EWMA recursive formula
    for index in range(observed_days, len(data)):
        data.loc[index, 'Volatility'] = np.sqrt(
            (1 - decay_factor) * data.loc[index, 'Log Return'] ** 2 + decay_factor * data.loc[(index - 1), 'Volatility'] ** 2)
    # drop the rows for null value
    data.dropna(inplace=True)
    data.reset_index(inplace=True, drop=True)

    return data

## Step2 Calculate One-Day VaR 

### 2.1 Monte Carlo Simulation

In [58]:
# use random draws from a hypothesized return distribution


def mc_simulation_var(data, observed_days, decay_factor, confidence_level):
    """
    data type: dataframe(dataframe contains date, price and log-return)
    observed_days type: int(observed days)
    decay_factor type: float(decay factor)
    confidence_level type: float(confidence_level for VaR)
    output type: dataframe(dataframe contains date, price, log-return and mc-simulation-var)
    """
    # calculate the EWMA volatility
    data = ewma_vol(data, observed_days, decay_factor)

    # use monte carlo simulation to create the var
    # calculate the var using the quantile of 1000 simulation
    for index in range(len(data)):
        random_returns = np.random.normal(
            loc=0, scale=data.loc[index, 'Volatility'], size=1000)
        var_return = np.percentile(
            random_returns, (1 - confidence_level) * 100)
        data.loc[index, 'VaR'] = data.loc[index,
                                          'Price'] * (1 - np.exp(var_return))

    return data

### 2.2 Parametric Method

In [59]:
# based on a hypothesized return distribution plus a volatility estimate


def parametric_var(data, observed_days, decay_factor, confidence_level):
    """
    data type: dataframe(dataframe contains date, price and log-return)
    observed_days type: int(observed days)
    decay_factor type: float(decay factor)
    confidence_level type: float(confidence_level for VaR)
    output type: dataframe(dataframe contains date, price, log-return and parametric-var)
    """
    # calculate the EWMA volatility
    data = ewma_vol(data, observed_days, decay_factor)
    # use parametric method to create the var
    data['VaR'] = data['Price'] * (1 - np.exp(norm.ppf(
        1 - confidence_level, loc=0, scale=data['Volatility'])))

    return data

### 2.3 Historical Method

In [60]:
# based on historical returns over some past observation period
# no distributional hypothesis


def historical_var(data, observed_days, confidence_level):
    """
    data type: dataframe(dataframe contains date, price and log-return)
    observed_days type: int(observed days)
    confidence_level type: float(confidence_level for VaR)
    output type: dataframe(dataframe contains date, price, log-return and historical-var)
    """

    for index in range(observed_days - 1, len(data)):
        data.loc[index, 'VaR'] = data.loc[index, 'Price'] * (1 - np.exp(np.percentile(
            data['Log Return'][index - observed_days + 1:index], (1 - confidence_level) * 100)))

    # drop the rows for null value
    data.dropna(inplace=True)
    data.reset_index(inplace=True, drop=True)

    return data

## Step3 Back-Tested VaR Model with Chi-Squared Test 

In [61]:
# use the Chi-Square goodness-of-fit test to back-test the VaR model


def backtest_var(data_var, confidence_level, confidence_level_chi2):
    """
    data type: dataframe(dataframe contains date, price, log-return and VaR)
    confidence_level type: float(confidence_interval for VaR)
    confidence_level_chi2 type: float(confidence_interval for chi-squared test)
    output type: dictionary(dictionary contains chi-squred test results)
    """
    # calculate the number of exceed observations
    exceed_num = len(
        data_var[(data_var['Price'].shift(1) - data_var['Price']) > data_var['VaR']])
    # calculate the total observations
    total_num = len(data_var) - 1
    # calculate the test statistics
    test_statistics = 2 * (np.log((exceed_num / total_num)**exceed_num * (1 - exceed_num / total_num)**(
        total_num - exceed_num)) - np.log((1 - confidence_level)**exceed_num * confidence_level**(total_num - exceed_num)))
    # calculate the 0.95 chi-squared statistics
    chi_squared_statistics = chi2.ppf(confidence_level_chi2, 1)

    # create the dictionnary to diaplay the results
    test_result = {}
    test_result['VaR Confidence-level'] = confidence_level
    test_result['Chi2 Confidence-level'] = confidence_level_chi2
    test_result['Exceed Num'] = exceed_num
    test_result['Total Num'] = total_num
    test_result['Test Statistics'] = test_statistics
    test_result['Chi-squared Statistics'] = chi_squared_statistics
    if test_statistics < chi_squared_statistics:
        test_result['Null Hypothesis'] = 'Accept'
    else:
        test_result['Null Hypothesis'] = 'Reject'

    return test_result